In [ ]:
from ssec_tutorials import OLMO_MODEL

In [ ]:
import panel as pn
from langchain.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager
from langchain_core.prompts import PromptTemplate

template = """<|user|>{contents}<|user|>"""
prompt = PromptTemplate.from_template(template)

pn.extension()

model_path = OLMO_MODEL

def callback(contents, user, instance):
    callback_handler = pn.chat.langchain.PanelCallbackHandler(instance, user='OLMo', avatar='🌳')
    # Not return the result at the end of the generation
    # this prevents the model from repeating the result
    callback_handler.on_llm_end = lambda response, *args, **kwargs: None
    # Callbacks support token-wise streaming
    callback_manager = CallbackManager([callback_handler])
    llm = LlamaCpp(
        model_path=str(model_path),
        callback_manager=callback_manager,
        temperature=0.8,
        max_tokens=512,
        verbose=False,
        echo=False
    )
    llm_chain = prompt | llm
    llm_chain.invoke({"contents": contents})

pn.chat.ChatInterface(callback=callback).servable()